In [25]:
!pip install tensorflow

In [26]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from pathlib import Path
import time
from tensorflow.keras.models import load_model

In [27]:
class PrepareCallbackConfig:
    tensorboard_root_log_dir = "artifacts/prepare_callbacks/tensorboard_log_dir"
    checkpoint_model_filepath = "artifacts/prepare_callbacks/checkpoint_dir/model.keras"

class PrepareCallback:
    def __init__(self):
        self.callback_config_obj = PrepareCallbackConfig()
    
    @property
    def create_tb_callbacks(self):
        try:
            timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
            tb_running_log_dir = os.path.join(
                self.callback_config_obj.tensorboard_root_log_dir,
                f"tb_logs_at_{timestamp}"
            )
            return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
        except Exception as e:
            # logging.info("Error in create_tb_callbacks")
            raise (e)
        
    @property
    def create_ckpt_callbacks(self):
        try:
            return tf.keras.callbacks.ModelCheckpoint(
                filepath = self.callback_config_obj.checkpoint_model_filepath,
                save_best_only = True
            )
 
        except Exception as e:
            # logging.info("Error in create_tb_callbacks")
            raise (e)
        
    def get_tb_ckpt_callbacks(self):
        try:
            return [
                self.create_tb_callbacks,
                self.create_ckpt_callbacks
            ]
        except Exception as e:
            # logging.info("Error in get_tb_ckpt_callbacks")
            raise (e)

In [74]:
class Training:
    def __init__(self):
        self.updated_base_model_path = "../artifacts/prepare_base_model/base_model_updated.h5"
        self.params_image_size = [224,224,3]
        self.params_batch_size = 16
        self.training_data = os.path.join("../artifacts/data_ingestion","Chicken-fecal-images")
        self.params_epochs = 5
        self.params_is_augmentation = True
        self.trained_model_path = "../artifacts/training/model.h5"
        self.learning_rate = 0.01
        

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.updated_base_model_path,
            compile = False
        )
        self.model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate = self.learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"]
        )

        self.model.summary()
##############################
        # # Load the base model without compiling
        # base_model = load_model(self.updated_base_model_path, compile=False)

        # # Define the input layer explicitly
        # inputs = tf.keras.Input(shape=(224, 224, 3))
        # outputs = base_model(inputs)

        # # Create a new model with the correct input shape
        # self.model = tf.keras.Model(inputs=inputs, outputs=outputs)
        # self.model.compile(
        #     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        #     loss='categorical_crossentropy',
        #     metrics=['accuracy']
        # )

######################################
        # self.model = tf.keras.models.load_model(
        #     self.updated_base_model_path
        # )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.params_image_size[:-1],
            batch_size = self.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )

        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

    def initiate_train(self, callback_list:list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.train_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path = self.trained_model_path,
            model = self.model
        )
    

In [76]:
callback_obj = PrepareCallback()
callback_list = callback_obj.get_tb_ckpt_callbacks()
training_obj = Training()
training_obj.get_base_model()
training_obj.train_valid_generator()
training_obj.initiate_train(
        callback_list=callback_list
    )

Model: "model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)                 │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │          50,178 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

Found 56 images belonging to 2 classes.
Found 232 images belonging to 2 classes.
Epoch 1/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.4993 - loss: 11.4465 - val_accuracy: 0.4167 - val_loss: 14.4417
Epoch 2/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.6250 - loss: 8.9146 - val_accuracy: 0.0000e+00 - val_loss: 10.0738
Epoch 3/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.5672 - loss: 9.7204 - val_accuracy: 0.4167 - val_loss: 18.5809
Epoch 4/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.3750 - loss: 19.5916 - val_accuracy: 0.0000e+00 - val_loss: 22.3310
Epoch 5/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.6656 - loss: 6.4951 - val_accuracy: 0.8333 - val_loss: 1.8632
